In [357]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from scrape_fbref import *

In [358]:
base_path = os.getcwd()
backend_root = os.path.dirname(base_path)

In [359]:
file_path = os.path.join(backend_root, "data/cleaned_merged_seasons.csv")
df = pd.read_csv(file_path)

/var/folders/yv/683h12gj4mdc5txzm0lh6j0h0000gn/T/ipykernel_4777/2866334842.py:2: DtypeWarning: Columns (2,3,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [360]:
df.columns

Index(['season_x', 'name', 'position', 'team_x', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'expected_goals', 'expected_assists',
       'expected_goals_conceded', 'GW'],
      dtype='object')

# PREPARE DATA

In [361]:
df['season_x'].unique()

array(['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22',
       '2022-23', '2023-24', '2024-25'], dtype=object)

In [362]:
for i in df.columns:
    print(f"{i}: {df[i].isna().sum()}")

season_x: 0
name: 0
position: 90496
team_x: 90496
assists: 0
bonus: 0
bps: 0
clean_sheets: 0
creativity: 0
element: 0
fixture: 0
goals_conceded: 0
goals_scored: 0
ict_index: 0
influence: 0
kickoff_time: 0
minutes: 0
opponent_team: 0
opp_team_name: 27614
own_goals: 0
penalties_missed: 0
penalties_saved: 0
red_cards: 0
round: 0
saves: 0
selected: 0
team_a_score: 59
team_h_score: 59
threat: 0
total_points: 0
transfers_balance: 0
transfers_in: 0
transfers_out: 0
value: 0
was_home: 0
yellow_cards: 0
expected_goals: 140308
expected_assists: 140308
expected_goals_conceded: 140308
GW: 0


In [363]:
df_25 = df[df['season_x'] == '2023-24']
for i in df_25.columns:
    print(f"{i}: {df_25[i].isna().sum()}")

season_x: 0
name: 0
position: 0
team_x: 0
assists: 0
bonus: 0
bps: 0
clean_sheets: 0
creativity: 0
element: 0
fixture: 0
goals_conceded: 0
goals_scored: 0
ict_index: 0
influence: 0
kickoff_time: 0
minutes: 0
opponent_team: 0
opp_team_name: 0
own_goals: 0
penalties_missed: 0
penalties_saved: 0
red_cards: 0
round: 0
saves: 0
selected: 0
team_a_score: 0
team_h_score: 0
threat: 0
total_points: 0
transfers_balance: 0
transfers_in: 0
transfers_out: 0
value: 0
was_home: 0
yellow_cards: 0
expected_goals: 0
expected_assists: 0
expected_goals_conceded: 0
GW: 0


In [364]:
df_25['fixture'].unique()

array([  7,   4,  10,   8,   3,   1,   6,   9,   2,   5,  17,  18,  20,
        13,  14,  16,  19,  11,  12,  30,  28,  24,  23,  22,  29,  27,
        25,  26,  21,  40,  31,  33,  32,  39,  36,  34,  38,  37,  35,
        48,  45,  47,  46,  42,  49,  50,  44,  41,  43,  60,  54,  53,
        59,  52,  58,  57,  55,  51,  56,  69,  64,  65,  61,  67,  62,
        70,  66,  15,  63,  68,  76,  73,  78,  72,  74,  75,  71,  79,
        80,  77,  89,  84,  86,  88,  83,  82,  87,  85,  90,  81,  91,
        95,  98,  94,  97,  93, 100,  92,  96,  99, 109, 110, 104, 103,
       108, 101, 106, 107, 102, 105, 114, 115, 118, 119, 117, 113, 120,
       111, 112, 116, 129, 127, 123, 128, 121, 126, 124, 122, 125, 130,
       134, 135, 138, 139, 133, 132, 137, 131, 136, 140, 147, 150, 142,
       145, 143, 141, 144, 149, 146, 148, 158, 154, 157, 152, 160, 156,
       159, 153, 155, 151, 165, 166, 161, 169, 163, 167, 168, 170, 164,
       171, 180, 179, 172, 177, 175, 173, 174, 178, 190, 186, 18

In [365]:
# Assuming your merged DataFrame is called df_merged and has columns 'season_x' and 'position'
for season in sorted(df['season_x'].unique()):
    season_mask = df['season_x'] == season
    n_total = season_mask.sum()
    n_nans = df.loc[season_mask, 'position'].isna().sum()
    percent_nans = (n_nans / n_total) * 100 if n_total > 0 else 0
    print(f"{season}: {percent_nans:.2f}% NaN values in position column")

2016-17: 100.00% NaN values in position column
2017-18: 100.00% NaN values in position column
2018-19: 100.00% NaN values in position column
2019-20: 100.00% NaN values in position column
2020-21: 0.00% NaN values in position column
2021-22: 0.00% NaN values in position column
2022-23: 0.00% NaN values in position column
2023-24: 0.00% NaN values in position column
2024-25: 0.00% NaN values in position column


In [366]:
# Load a fixture file to see the difficulty ratings
import os

# Check if fixture files exist and examine structure
fixture_file = "/Users/bragehs/Documents/FPL helper/Fantasy-Premier-League/data/2023-24/fixtures.csv"
if os.path.exists(fixture_file):
    fixtures_df = pd.read_csv(fixture_file)
    print("Fixtures file columns:")
    print(fixtures_df.columns.tolist())
    print("\nSample fixture difficulty data:")
    print(fixtures_df[['id', 'team_h', 'team_a', 'team_h_difficulty', 'team_a_difficulty']].head())
    print(f"\nDifficulty ratings range: {fixtures_df['team_h_difficulty'].min()} to {fixtures_df['team_h_difficulty'].max()}")
else:
    print("Fixture file not found")

Fixtures file columns:
['code', 'event', 'finished', 'finished_provisional', 'id', 'kickoff_time', 'minutes', 'provisional_start_time', 'started', 'team_a', 'team_a_score', 'team_h', 'team_h_score', 'stats', 'team_h_difficulty', 'team_a_difficulty', 'pulse_id']

Sample fixture difficulty data:
   id  team_h  team_a  team_h_difficulty  team_a_difficulty
0   1       6      13                  5                  2
1   2       1      16                  2                  5
2   3       3      19                  2                  2
3   4       5      12                  2                  3
4   5       9      10                  2                  2

Difficulty ratings range: 2 to 5


In [367]:
def add_fixture_difficulty_to_dataframe(df):
    """
    Add fixture difficulty ratings to the main dataframe by merging with fixture files
    from each season directory.
    """
    base_path = backend_root + "/data/"
    
    # Get all seasons from the dataframe
    seasons = df['season_x'].unique()
    print(f"Found seasons in data: {sorted(seasons)}")
    
    # Store all fixture data
    all_fixtures = []
    
    for season in seasons:
        fixture_file = os.path.join(base_path, season, "fixtures.csv")
        
        if os.path.exists(fixture_file):
            try:
                season_fixtures = pd.read_csv(fixture_file)
                
                # Add season identifier
                season_fixtures['fixture_season'] = season
                
                # Select relevant columns and rename to avoid conflicts
                season_fixtures = season_fixtures[['id', 'team_h_difficulty', 'team_a_difficulty', 'fixture_season']]
                season_fixtures = season_fixtures.rename(columns={'id': 'fixture_id'})
                
                all_fixtures.append(season_fixtures)
                print(f"✓ Loaded {len(season_fixtures)} fixtures from {season}")
                
            except Exception as e:
                print(f"✗ Error loading fixtures from {season}: {e}")
        else:
            print(f"✗ Fixture file not found for {season}: {fixture_file}")
    
    if not all_fixtures:
        print("No fixture files found!")
        return df
    
    # Combine all fixture data
    combined_fixtures = pd.concat(all_fixtures, ignore_index=True)
    print(f"\nTotal fixtures loaded: {len(combined_fixtures)}")
    
    # Merge with main dataframe
    # The 'fixture' column in df corresponds to 'fixture_id' in fixtures
    df_with_difficulty = df.merge(
        combined_fixtures,
        left_on=['fixture', 'season_x'],
        right_on=['fixture_id', 'fixture_season'],
        how='left'
    )
    
    # Create player-specific difficulty rating
    # If player was home team, use team_h_difficulty, else use team_a_difficulty
    df_with_difficulty['fixture_difficulty'] = df_with_difficulty.apply(
        lambda row: row['team_h_difficulty'] if row['was_home'] else row['team_a_difficulty'],
        axis=1
    )
    
    # Drop temporary columns that were created during merge
    columns_to_drop = ['fixture_id', 'fixture_season']
    existing_columns_to_drop = [col for col in columns_to_drop if col in df_with_difficulty.columns]
    
    if existing_columns_to_drop:
        df_with_difficulty = df_with_difficulty.drop(existing_columns_to_drop, axis=1)
    
    print(f"\nMerge results:")
    print(f"Original dataframe: {len(df)} rows")
    print(f"With difficulty: {len(df_with_difficulty)} rows")
    print(f"Missing difficulty values: {df_with_difficulty['fixture_difficulty'].isna().sum()}")
    
    return df_with_difficulty

# Apply the function to add fixture difficulty
print("Adding fixture difficulty ratings...")
df_enhanced = add_fixture_difficulty_to_dataframe(df)

# Check the results
print("\nFixture difficulty distribution:")
print(df_enhanced['fixture_difficulty'].value_counts().sort_index())

print("\nSample data with difficulty ratings:")
sample_with_difficulty = df_enhanced[['name', 'GW', 'season_x', 'was_home', 'opp_team_name', 
                                     'team_h_difficulty', 'team_a_difficulty', 'fixture_difficulty']].head(10)
print(sample_with_difficulty)

Adding fixture difficulty ratings...
Found seasons in data: ['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24', '2024-25']
✗ Fixture file not found for 2016-17: /Users/bragehs/Documents/FPL_forecast/backend/data/2016-17/fixtures.csv
✗ Fixture file not found for 2017-18: /Users/bragehs/Documents/FPL_forecast/backend/data/2017-18/fixtures.csv
✓ Loaded 380 fixtures from 2018-19
✓ Loaded 380 fixtures from 2019-20
✓ Loaded 380 fixtures from 2020-21
✓ Loaded 380 fixtures from 2021-22
✓ Loaded 380 fixtures from 2022-23
✓ Loaded 380 fixtures from 2023-24
✓ Loaded 380 fixtures from 2024-25

Total fixtures loaded: 2660

Merge results:
Original dataframe: 224152 rows
With difficulty: 224152 rows
Missing difficulty values: 46146

Fixture difficulty distribution:
fixture_difficulty
1.0     2772
2.0    71985
3.0    52633
4.0    38563
5.0    12053
Name: count, dtype: int64

Sample data with difficulty ratings:
                 name  GW season_x  was_home opp_team_

In [368]:
for season in sorted(df_enhanced['season_x'].unique()):
    season_mask = df_enhanced['season_x'] == season
    n_total = season_mask.sum()
    n_nans = df_enhanced.loc[season_mask, 'fixture_difficulty'].isna().sum()
    percent_nans = (n_nans / n_total) * 100 if n_total > 0 else 0
    print(f"{season}: {percent_nans:.2f}% NaN values in fixture_difficulty column")

2016-17: 100.00% NaN values in fixture_difficulty column
2017-18: 100.00% NaN values in fixture_difficulty column
2018-19: 0.00% NaN values in fixture_difficulty column
2019-20: 0.00% NaN values in fixture_difficulty column
2020-21: 0.00% NaN values in fixture_difficulty column
2021-22: 0.00% NaN values in fixture_difficulty column
2022-23: 0.00% NaN values in fixture_difficulty column
2023-24: 0.00% NaN values in fixture_difficulty column
2024-25: 0.00% NaN values in fixture_difficulty column


In [369]:
df_enhanced.head()

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,...,value,was_home,yellow_cards,expected_goals,expected_assists,expected_goals_conceded,GW,team_h_difficulty,team_a_difficulty,fixture_difficulty
0,2016-17,Aaron Cresswell,NaN,NaN,0,0,0,0,0.0,454,...,55,False,0,NaN,NaN,NaN,1,NaN,NaN,NaN
1,2016-17,Aaron Lennon,NaN,NaN,0,0,6,0,0.3,142,...,60,True,0,NaN,NaN,NaN,1,NaN,NaN,NaN
2,2016-17,Aaron Ramsey,NaN,NaN,0,0,5,0,4.9,16,...,80,True,0,NaN,NaN,NaN,1,NaN,NaN,NaN
3,2016-17,Abdoulaye Doucouré,NaN,NaN,0,0,0,0,0.0,482,...,50,False,0,NaN,NaN,NaN,1,NaN,NaN,NaN
4,2016-17,Abdul Rahman Baba,NaN,NaN,0,0,0,0,0.0,80,...,55,True,0,NaN,NaN,NaN,1,NaN,NaN,NaN


In [370]:
print(len(df_enhanced))

224152


In [371]:
remove_season = df_enhanced[df_enhanced['season_x'].isin(['2024-25', '2023-24', '2022-23'])].copy() #only seasons with xG
print(len(remove_season))
testing = remove_season[["name", "opp_team_name", "team_h_difficulty", "team_a_difficulty", "fixture_difficulty"]].head(20)
testing

83844


,name,opp_team_name,team_h_difficulty,team_a_difficulty,fixture_difficulty
140308,Nathan Redmond,Spurs,2.0,4.0,4.0
140309,Junior Stanislas,Aston Villa,2.0,2.0,2.0
140310,Armando Broja,Everton,3.0,2.0,2.0
140311,Fabian Schär,Nott'm Forest,2.0,4.0,2.0
140312,Jonny Evans,Brentford,3.0,3.0,3.0
140313,Brennan Johnson,Newcastle,2.0,4.0,4.0
140314,Cheick Doucouré,Arsenal,4.0,2.0,4.0
140315,Oliver Hammond,Newcastle,2.0,4.0,4.0
140316,Luke Cundle,Leeds,2.0,2.0,2.0
140317,Fin Stevens,Leicester,3.0,3.0,3.0


In [372]:
remove_season.columns

Index(['season_x', 'name', 'position', 'team_x', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'expected_goals', 'expected_assists',
       'expected_goals_conceded', 'GW', 'team_h_difficulty',
       'team_a_difficulty', 'fixture_difficulty'],
      dtype='object')

In [373]:
for i in remove_season.columns:
    print(f"{i}: {remove_season[i].isna().sum()}")

season_x: 0
name: 0
position: 0
team_x: 0
assists: 0
bonus: 0
bps: 0
clean_sheets: 0
creativity: 0
element: 0
fixture: 0
goals_conceded: 0
goals_scored: 0
ict_index: 0
influence: 0
kickoff_time: 0
minutes: 0
opponent_team: 0
opp_team_name: 27614
own_goals: 0
penalties_missed: 0
penalties_saved: 0
red_cards: 0
round: 0
saves: 0
selected: 0
team_a_score: 0
team_h_score: 0
threat: 0
total_points: 0
transfers_balance: 0
transfers_in: 0
transfers_out: 0
value: 0
was_home: 0
yellow_cards: 0
expected_goals: 0
expected_assists: 0
expected_goals_conceded: 0
GW: 0
team_h_difficulty: 0
team_a_difficulty: 0
fixture_difficulty: 0


In [374]:
print(remove_season['GW'].unique())
print(remove_season['round'].unique())

[ 1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38  7]
[ 1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38  7]


In [375]:
remove_season['opponent_team'].unique()

array([18,  2,  8, 16,  4, 15,  1, 11, 10, 17, 13,  9, 20, 12, 14, 19,  5,
        6,  7,  3])

In [376]:
remove_season.columns

Index(['season_x', 'name', 'position', 'team_x', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'expected_goals', 'expected_assists',
       'expected_goals_conceded', 'GW', 'team_h_difficulty',
       'team_a_difficulty', 'fixture_difficulty'],
      dtype='object')

In [377]:
remove_season['bps'].describe()

count    83844.000000
mean         4.839702
std          9.109689
min        -25.000000
25%          0.000000
50%          0.000000
75%          6.000000
max        102.000000
Name: bps, dtype: float64

In [379]:
remove_season[remove_season['name'] == 'Mohamed Salah']['element'].unique()

array([283, 308, 328])

In [380]:
remove_season['season_x'].unique()

array(['2022-23', '2023-24', '2024-25'], dtype=object)

In [381]:
remove_season.GW.unique()

array([ 1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38,  7])

In [382]:
remove_season.loc[remove_season['position'].isin(['GK', 'GKP']), 'position'] = 'GK'

In [383]:
remove_season.loc[remove_season['fixture_difficulty'] == 1, 'fixture_difficulty'] = 2

In [384]:
remove_season['position'].unique()

array(['MID', 'FWD', 'DEF', 'GK', 'AM'], dtype=object)

In [385]:
remove_season[remove_season['position'].isin(['GK', 'MID', 'DEF', 'FWD'])]

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,...,value,was_home,yellow_cards,expected_goals,expected_assists,expected_goals_conceded,GW,team_h_difficulty,team_a_difficulty,fixture_difficulty
140308,2022-23,Nathan Redmond,MID,Southampton,0,0,3,0,0.0,403,...,55,False,0,0.0,0.0,0.0,1,2.0,4.0,4.0
140309,2022-23,Junior Stanislas,MID,Bournemouth,0,0,3,0,0.0,58,...,50,True,0,0.0,0.0,0.0,1,2.0,2.0,2.0
140310,2022-23,Armando Broja,FWD,Chelsea,0,0,3,0,0.3,150,...,55,False,0,0.0,0.0,0.0,1,3.0,2.0,2.0
140311,2022-23,Fabian Schär,DEF,Newcastle,0,3,43,1,14.6,366,...,45,True,0,0.0,0.0,0.0,1,2.0,4.0,2.0
140312,2022-23,Jonny Evans,DEF,Leicester,0,0,15,0,1.3,249,...,45,True,0,0.0,0.0,0.0,1,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224146,2024-25,Diego Carlos Santos Silva,DEF,Aston Villa,0,0,0,0,0.0,35,...,45,False,0,0.0,0.0,0.0,38,3.0,3.0,3.0
224147,2024-25,Đorđe Petrović,GK,Chelsea,0,0,0,0,0.0,183,...,44,False,0,0.0,0.0,0.0,38,3.0,4.0,4.0
224149,2024-25,Hannibal Mejbri,MID,Man Utd,0,0,0,0,0.0,373,...,45,True,0,0.0,0.0,0.0,38,3.0,3.0,3.0
224150,2024-25,Callum Bates,MID,Everton,0,0,0,0,0.0,678,...,45,False,0,0.0,0.0,0.0,38,3.0,4.0,4.0


In [386]:
remove_season['fixture_difficulty'].unique()

array([4., 2., 3., 5.])

In [387]:
#remove rows where minutes is 0
remove_season = remove_season[remove_season['minutes'] > 0].copy()

In [388]:
def add_rolling_features(df, windows=[3, 5]):
    """
    Add rolling statistics for key performance metrics
    """
    df_enhanced = df.copy()
    
    # Group by player and season for rolling calculations
    for window in windows:
        # Performance rolling averages
        df_enhanced[f'total_points_rolling_{window}'] = df_enhanced.groupby(['element', 'season_x'])['total_points'].transform(lambda x: x.rolling(window, min_periods=1).mean())
        
        # Form indicators
        df_enhanced[f'goals_rolling_{window}'] = df_enhanced.groupby(['element', 'season_x'])['goals_scored'].transform(lambda x: x.rolling(window, min_periods=1).mean())
        df_enhanced[f'assists_rolling_{window}'] = df_enhanced.groupby(['element', 'season_x'])['assists'].transform(lambda x: x.rolling(window, min_periods=1).mean())
        df_enhanced[f'minutes_rolling_{window}'] = df_enhanced.groupby(['element', 'season_x'])['minutes'].transform(lambda x: x.rolling(window, min_periods=1).mean())
        
        # Consistency metrics
        df_enhanced[f'points_std_{window}'] = df_enhanced.groupby(['element', 'season_x'])['total_points'].transform(lambda x: x.rolling(window, min_periods=1).std().fillna(0))
        
        # Clean sheets for defenders/keepers
        df_enhanced[f'clean_sheets_rolling_{window}'] = df_enhanced.groupby(['element', 'season_x'])['clean_sheets'].transform(lambda x: x.rolling(window, min_periods=1).mean())
    
    return df_enhanced

def add_streak_features(df):
    """
    Add streak-based features
    """
    df_enhanced = df.copy()
    
    # Points streaks
    df_enhanced['points_streak'] = df_enhanced.groupby(['element', 'season_x']).apply(
        lambda x: x['total_points'].ge(x['total_points'].median()).groupby(
            (~x['total_points'].ge(x['total_points'].median())).cumsum()
        ).cumsum()
    ).values
    
    # Games without return (goals/assists)
    df_enhanced['games_without_return'] = df_enhanced.groupby(['element', 'season_x']).apply(
        lambda x: ((x['goals_scored'] + x['assists']) == 0).groupby(
            ((x['goals_scored'] + x['assists']) > 0).cumsum()
        ).cumsum()
    ).values
    
    # Minutes played consistency
    df_enhanced['starts_streak'] = df_enhanced.groupby(['element', 'season_x']).apply(
        lambda x: (x['minutes'] >= 60).groupby(
            (x['minutes'] < 60).cumsum()
        ).cumsum()
    ).values
    
    return df_enhanced

In [389]:
remove_season['season_progress'] = remove_season['GW'] / remove_season['GW'].max()
#remove_season = add_rolling_features(remove_season, windows=[3, 5])
#remove_season = add_streak_features(remove_season)

In [390]:
remove_season.columns

Index(['season_x', 'name', 'position', 'team_x', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'expected_goals', 'expected_assists',
       'expected_goals_conceded', 'GW', 'team_h_difficulty',
       'team_a_difficulty', 'fixture_difficulty', 'season_progress'],
      dtype='object')

In [391]:
# Assume df is your DataFrame with 'season_x' and 'GW' columns
# Sort by season and GW
remove_season = remove_season.sort_values(['season_x', 'GW']).reset_index(drop=True)

In [392]:
remove_season.iloc[-1]

season_x                                 2024-25
name                       Jefferson Lerma Solís
position                                     MID
team_x                            Crystal Palace
assists                                        0
bonus                                          0
bps                                           15
clean_sheets                                   0
creativity                                   4.1
element                                      206
fixture                                      374
goals_conceded                                 1
goals_scored                                   0
ict_index                                    3.6
influence                                   30.2
kickoff_time                2025-05-25T15:00:00Z
minutes                                       90
opponent_team                                 12
opp_team_name                                NaN
own_goals                                      0
penalties_missed    

In [393]:
# Calculate split indices
n = len(remove_season)
train_end = int(0.7 * n)
val_end = int(0.85 * n)

train_df = remove_season.iloc[:train_end:]
val_df = remove_season.iloc[train_end:val_end]
test_df = remove_season.iloc[val_end:]

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

Train: 24007, Val: 5144, Test: 5145


In [409]:
categorical_features = [
    'position',           # Player position (GK, DEF, MID, FWD)
]

binary_features = [
    'was_home',          # Home/Away (boolean)
]

min_max_features = [
    'bonus', 'minutes','fixture_difficulty', 'yellow_cards', 'red_cards', 
    'own_goals', 'saves', 'goals_conceded','clean_sheets', 'penalties_missed',
    'penalties_saved', 'assists', 'goals_scored', 'season_progress', 'expected_goals',
    'expected_assists', 'expected_goals_conceded','creativity', 'influence',
     'threat', 'ict_index', 'bps',
    # Rolling features
   # 'total_points_rolling_3', 'total_points_rolling_5',
    #'goals_rolling_3', 'goals_rolling_5',
   # 'assists_rolling_3', 'assists_rolling_5',
   # 'minutes_rolling_3', 'minutes_rolling_5',
   # 'points_std_3', 'points_std_5',
   # 'clean_sheets_rolling_3', 'clean_sheets_rolling_5',

    # Streak features
   # 'points_streak', 'games_without_return', 'starts_streak',
    

]


metadata_features = [
    'name',                    # Player name (use for grouping/analysis)
    'element',                 # Player ID (use for grouping sequences)
    'team_x',                 
    'season_x', 
]
time_feature = 'round'
target = 'total_points'  # Target variable for prediction

In [410]:
def preprocess_data(df, scalers=None, encoders=None, fit=False):
    """
    Preprocess the dataframe by scaling continuous features and encoding categorical features.
    
    Args:
        df: DataFrame to process
        scalers: Dictionary of fitted scalers (if fit=False)
        encoders: Dictionary of fitted encoders (if fit=False)
        fit: Whether to fit new scalers/encoders or use existing ones
    
    Returns:
        X_processed: Processed DataFrame
        scalers: Dictionary of scalers (if fit=True)
        encoders: Dictionary of encoders (if fit=True)
    """
    # Separate features
    X_min_max = df[min_max_features]
    X_categorical = df[categorical_features]
    X_metadata = df[metadata_features]

    
    if fit:
        # Fit new scalers and encoders
        scalers = {}
        encoders = {}
        
        # Scale continuous features
        scalers['continuous'] = MinMaxScaler()
        X_minmax_scaled = scalers['continuous'].fit_transform(X_min_max)
        
        # Encode categorical features
        encoders['categorical'] = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        X_categorical_encoded = encoders['categorical'].fit_transform(X_categorical)
        
    else:
        # Use existing scalers and encoders
        X_minmax_scaled = scalers['continuous'].transform(X_min_max)
        X_categorical_encoded = encoders['categorical'].transform(X_categorical)
    
    # Extract time and target
    X_time = df['GW']
    y = df[target]
    
    # Combine processed features
    X_time_df = pd.DataFrame(X_time.values, columns=['GW'])
    X_cont_df = pd.DataFrame(X_minmax_scaled, columns=min_max_features)
    X_cat_df = pd.DataFrame(X_categorical_encoded, 
                           columns=encoders['categorical'].get_feature_names_out(categorical_features))
    X_metadata_df = pd.DataFrame(X_metadata.values, columns=metadata_features)
    target_df = pd.DataFrame(y.values, columns=[target])
    
    X_processed = pd.concat([X_time_df, X_cont_df, X_cat_df, X_metadata_df, 
                             target_df], axis=1)
    
    if fit:
        return X_processed, scalers, encoders
    else:
        return X_processed

# Fit scalers and encoders on training data only
processed_train_df, scalers, encoders = preprocess_data(train_df, fit=True)

# Transform validation and test data using the fitted scalers/encoders
processed_val_df = preprocess_data(val_df, scalers=scalers, encoders=encoders, fit=False)

processed_test_df = preprocess_data(test_df, scalers=scalers, encoders=encoders, fit=False)

In [411]:
for i in processed_train_df.columns:
    print(f"{i}: {processed_val_df[i].isna().sum()}")

GW: 0
bonus: 0
minutes: 0
fixture_difficulty: 0
yellow_cards: 0
red_cards: 0
own_goals: 0
saves: 0
goals_conceded: 0
clean_sheets: 0
penalties_missed: 0
penalties_saved: 0
assists: 0
goals_scored: 0
season_progress: 0
expected_goals: 0
expected_assists: 0
expected_goals_conceded: 0
creativity: 0
influence: 0
threat: 0
ict_index: 0
bps: 0
position_DEF: 0
position_FWD: 0
position_GK: 0
position_MID: 0
name: 0
element: 0
team_x: 0
season_x: 0
total_points: 0


In [412]:
remove_season[(remove_season['name'] == 'Mohamed Salah') & (remove_season['goals_scored'] == 2)]['total_points'].unique()

array([15, 21, 14, 16, 13, 18])

In [413]:
processed_train_df.columns

Index(['GW', 'bonus', 'minutes', 'fixture_difficulty', 'yellow_cards',
       'red_cards', 'own_goals', 'saves', 'goals_conceded', 'clean_sheets',
       'penalties_missed', 'penalties_saved', 'assists', 'goals_scored',
       'season_progress', 'expected_goals', 'expected_assists',
       'expected_goals_conceded', 'creativity', 'influence', 'threat',
       'ict_index', 'bps', 'position_DEF', 'position_FWD', 'position_GK',
       'position_MID', 'name', 'element', 'team_x', 'season_x',
       'total_points'],
      dtype='object')

In [414]:
processed_train_df.head()

,GW,bonus,minutes,fixture_difficulty,yellow_cards,red_cards,own_goals,saves,goals_conceded,clean_sheets,...,bps,position_DEF,position_FWD,position_GK,position_MID,name,element,team_x,season_x,total_points
0,1,0.0,0.000000,0.666667,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.195122,0.0,0.0,0.0,1.0,Nathan Redmond,403,Southampton,2022-23,1
1,1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.195122,0.0,0.0,0.0,1.0,Junior Stanislas,58,Bournemouth,2022-23,1
2,1,0.0,0.157303,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.195122,0.0,1.0,0.0,0.0,Armando Broja,150,Chelsea,2022-23,1
3,1,1.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.0,...,0.520325,1.0,0.0,0.0,0.0,Fabian Schär,366,Newcastle,2022-23,15
4,1,0.0,1.000000,0.333333,0.0,0.0,0.0,0.0,0.222222,0.0,...,0.292683,1.0,0.0,0.0,0.0,Jonny Evans,249,Leicester,2022-23,1


In [415]:
print(processed_train_df['season_x'].unique())
print(processed_val_df['season_x'].unique())
print(processed_test_df['season_x'].unique())

['2022-23' '2023-24' '2024-25']
['2024-25']
['2024-25']


In [416]:
for season, group in processed_train_df.groupby('season_x'):
    for i in processed_train_df['element'].unique():
        unique_names = group[group['element'] == i]['name'].unique()
        if len(unique_names) > 1:
            print(f"{season}: {unique_names}")

#element is only a player ID within a season

2023-24: ['Yegor Yarmolyuk' 'Yegor Yarmoliuk']
2023-24: ['Michale Olakigbe' 'Michael Olakigbe']
2023-24: ['Djordje Petrovic' 'Đorđe Petrović']


In [417]:
print(len(processed_train_df))

24007


In [418]:
processed_train_df.columns

Index(['GW', 'bonus', 'minutes', 'fixture_difficulty', 'yellow_cards',
       'red_cards', 'own_goals', 'saves', 'goals_conceded', 'clean_sheets',
       'penalties_missed', 'penalties_saved', 'assists', 'goals_scored',
       'season_progress', 'expected_goals', 'expected_assists',
       'expected_goals_conceded', 'creativity', 'influence', 'threat',
       'ict_index', 'bps', 'position_DEF', 'position_FWD', 'position_GK',
       'position_MID', 'name', 'element', 'team_x', 'season_x',
       'total_points'],
      dtype='object')

In [419]:
processed_train_df['total_points'].describe()

count    24007.000000
mean         2.763652
std          2.955000
min         -4.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         26.000000
Name: total_points, dtype: float64

In [420]:
processed_train_df['assists_rolling_5'].describe()

KeyError: 'assists_rolling_5'

In [421]:
def create_sequences(df, past_sequences=5, future_sequences=3, min_sequences=1):
    """
    Create sequences of data for each player based on the Gameweek (GW).
    Supports variable-length sequences with left-padding for flexibility.
    
    Args:
        df: DataFrame with player data
        past_sequences: Maximum number of past gameweeks to use (default: 5)
        future_sequences: Number of future gameweeks to predict (default: 3)
        min_sequences: Minimum number of past gameweeks required (default: 1)
    
    Returns:
        X_tensor: Input sequences with left-padding for shorter sequences
        y_tensor: Target sequences
        future_fd_tensor: Future fixture difficulty sequences
    """
    # Sort to ensure features are always in same order
    feature_cols = sorted([col for col in df.columns if col not in ['total_points', 'GW', 'element', 'name', 'season_x', 'team_x']])
    X_seq, y_seq = [], []
    
    num_features = len(feature_cols)

    for player_id, stats in df.groupby(['element', 'season_x']):
        group = stats.sort_values('GW').reset_index(drop=True)
        
        # Iterate through the group to create sequences
        for i in range(min_sequences - 1, len(group) - future_sequences):
            # Determine the actual number of past gameweeks available for this sequence
            actual_past_available = min(i + 1, past_sequences)
            
            # Start index for actual sequence data
            start_idx_actual = i - actual_past_available + 1
            
            sequence_data = group.iloc[start_idx_actual : i + 1][feature_cols].values
            
            # Create the final sequence
            if len(sequence_data) < past_sequences:
                # Need padding - create padded sequence with zeros on the left
                padded_sequence = np.zeros((past_sequences, num_features))
                padding_needed = past_sequences - len(sequence_data)
                padded_sequence[padding_needed:] = sequence_data
            else:
                # No padding needed - use the last 'past_sequences' rows
                padded_sequence = sequence_data[-past_sequences:]
            
            # Get target and future fixture difficulty
            target_start_idx = i + 1
            target = group.iloc[target_start_idx : target_start_idx + future_sequences]['total_points'].values
            
            # Ensure target and future_fd have the correct length
            if len(target) != future_sequences:
                continue
                
            X_seq.append(padded_sequence)
            y_seq.append(target)
    
    # Convert to numpy arrays
    X_np = np.array(X_seq)
    y_np = np.array(y_seq)

    # Convert to tensors
    X_tensor = torch.tensor(X_np, dtype=torch.float32)
    y_tensor = torch.tensor(y_np, dtype=torch.float32)

    return X_tensor, y_tensor
        
    

In [354]:
# Test the flexible sequence creation with different minimum sequences
print("Testing flexible sequence creation with padding:")
print("-" * 50)

# Test with minimum 1 sequence (allows 1, 2, 3, 4, 5 gameweeks)
X_test, y_test = create_sequences(
    processed_train_df, 
    past_sequences=5, 
    future_sequences=1, 
    min_sequences=1
)

print(f"With min_sequences=1: {X_test.shape[0]} sequences created")

# Test with minimum 3 sequences (allows 3, 4, 5 gameweeks)
X_test_min3, y_test_min3 = create_sequences(
    processed_train_df, 
    past_sequences=5, 
    future_sequences=1, 
    min_sequences=3
)

print(f"With min_sequences=3: {X_test_min3.shape[0]} sequences created")

# Show example of padding - check if first sequence has zeros
print("\nExample of left-padding:")
print("First sequence shape:", X_test[0].shape)
print("First few rows of first sequence:")
print(X_test[0][:3, :5])  # Show first 3 timesteps, first 5 features
print("...")
print("Last few rows of first sequence:")
print(X_test[0][-3:, :5])  # Show last 3 timesteps, first 5 features

# Check if any rows are all zeros (indicating padding)
zero_rows = (X_test[0] == 0).all(axis=1)
print(f"Rows that are all zeros (padding): {zero_rows.sum()}")
print(f"Padding positions: {np.where(zero_rows)[0].tolist()}")

Testing flexible sequence creation with padding:
--------------------------------------------------
With min_sequences=1: 22472 sequences created
With min_sequences=3: 19784 sequences created

Example of left-padding:
First sequence shape: torch.Size([5, 41])
First few rows of first sequence:
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
...
Last few rows of first sequence:
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1870]])
Rows that are all zeros (padding): 4
Padding positions: [0, 1, 2, 3]


In [355]:
# Analyze padding distribution in the created sequences
print("Analyzing padding distribution in sequences:")
print("=" * 50)

def analyze_padding_distribution(X_tensor, tensor_name=""):
    """
    Analyze how much padding is in each sequence
    """
    print(f"\n{tensor_name} Analysis:")
    print(f"Total sequences: {X_tensor.shape[0]}")
    
    padding_counts = {}
    
    for i in range(X_tensor.shape[0]):
        sequence = X_tensor[i]
        
        # Count rows that are all zeros (padding)
        zero_rows = (sequence == 0).all(axis=1)
        num_padding = zero_rows.sum().item()
        
        # Count actual data rows
        actual_data_rows = X_tensor.shape[1] - num_padding
        
        if actual_data_rows not in padding_counts:
            padding_counts[actual_data_rows] = 0
        padding_counts[actual_data_rows] += 1
    
    # Sort by number of actual data rows
    sorted_counts = sorted(padding_counts.items())
    
    print("\nSequence length distribution:")
    print("Actual data rows | Count | Percentage | Padding rows")
    print("-" * 55)
    
    for actual_rows, count in sorted_counts:
        padding_rows = X_tensor.shape[1] - actual_rows
        percentage = (count / X_tensor.shape[0]) * 100
        print(f"{actual_rows:14d} | {count:5d} | {percentage:8.1f}% | {padding_rows:12d}")
    
    # Summary statistics
    total_padded = sum(count for actual_rows, count in sorted_counts if actual_rows < X_tensor.shape[1])
    total_unpadded = padding_counts.get(X_tensor.shape[1], 0)
    
    print(f"\nSummary:")
    print(f"Sequences with NO padding (full 5 gameweeks): {total_unpadded:,}")
    print(f"Sequences with padding (1-4 gameweeks): {total_padded:,}")
    print(f"Percentage with padding: {(total_padded / X_tensor.shape[0]) * 100:.1f}%")
    
    return padding_counts

# Analyze both test sequences
test_padding = analyze_padding_distribution(X_test, "Test sequences (min_sequences=1)")
test_min3_padding = analyze_padding_distribution(X_test_min3, "Test sequences (min_sequences=3)")

# Let's also check a few actual examples
print("\n" + "=" * 50)
print("Example sequences with different padding levels:")
print("=" * 50)

# Find examples of different padding levels
examples_found = {}
for i in range(min(100, X_test.shape[0])):  # Check first 100 sequences
    sequence = X_test[i]
    zero_rows = (sequence == 0).all(axis=1)
    num_padding = zero_rows.sum().item()
    actual_data_rows = X_test.shape[1] - num_padding
    
    if actual_data_rows not in examples_found and len(examples_found) < 4:
        examples_found[actual_data_rows] = i

# Show examples
for actual_rows in sorted(examples_found.keys()):
    seq_idx = examples_found[actual_rows]
    padding_rows = X_test.shape[1] - actual_rows
    
    print(f"\nExample sequence with {actual_rows} actual data rows ({padding_rows} padding rows):")
    print(f"Sequence index: {seq_idx}")
    print("Shape:", X_test[seq_idx].shape)
    print("First 3 rows (should be padding if padding_rows > 0):")
    print(X_test[seq_idx][:3, :5])  # Show first 5 features
    print("Last 2 rows (should be actual data):")
    print(X_test[seq_idx][-2:, :5])  # Show first 5 features
    
    # Check if padding is where we expect it
    zero_rows = (X_test[seq_idx] == 0).all(axis=1)
    print(f"Zero rows (padding): {zero_rows.numpy()}")

Analyzing padding distribution in sequences:

Test sequences (min_sequences=1) Analysis:
Total sequences: 22472

Sequence length distribution:
Actual data rows | Count | Percentage | Padding rows
-------------------------------------------------------
             1 |  1391 |      6.2% |            4
             2 |  1297 |      5.8% |            3
             3 |  1205 |      5.4% |            2
             4 |   991 |      4.4% |            1
             5 | 17588 |     78.3% |            0

Summary:
Sequences with NO padding (full 5 gameweeks): 17,588
Sequences with padding (1-4 gameweeks): 4,884
Percentage with padding: 21.7%

Test sequences (min_sequences=3) Analysis:
Total sequences: 19784

Sequence length distribution:
Actual data rows | Count | Percentage | Padding rows
-------------------------------------------------------
             3 |  1205 |      6.1% |            2
             4 |   991 |      5.0% |            1
             5 | 17588 |     88.9% |            0



In [422]:
X_train, y_train = create_sequences(
    processed_train_df, 
    past_sequences=5, 
    future_sequences=1, 
    min_sequences=1 
)
X_val, y_val = create_sequences(
    processed_val_df, 
    past_sequences=5, 
    future_sequences=1, 
    min_sequences=1
)

print(f"Train sequences: {X_train.shape}, Targets: {y_train.shape}")
print(f"Val sequences: {X_val.shape}, Targets: {y_val.shape}")

Train sequences: torch.Size([22472, 5, 26]), Targets: torch.Size([22472, 1])
Val sequences: torch.Size([4659, 5, 26]), Targets: torch.Size([4659, 1])


In [424]:
torch.save((X_train, y_train), 'data/train_sequences.pt')
torch.save((X_val, y_val), 'data/val_sequences.pt')

print("Data preparation complete. Sequences saved to disk.")

Data preparation complete. Sequences saved to disk.


In [115]:
def create_inference_sequence(player_data, feature_cols, max_sequences=5):
    """
    Create a single padded sequence for inference when you have variable amounts of historical data.
    
    Args:
        player_data: DataFrame with player's historical data (sorted by GW)
        feature_cols: List of feature columns to include
        max_sequences: Maximum sequence length (will pad to this length)
    
    Returns:
        padded_sequence: numpy array with shape (max_sequences, num_features)
    """
    # Get the feature data
    sequence_data = player_data[feature_cols].values
    num_features = len(feature_cols)
    
    # Create padded sequence with zeros on the left
    padded_sequence = np.zeros((max_sequences, num_features))
    
    # Determine how much padding is needed
    actual_length = min(len(sequence_data), max_sequences)
    padding_needed = max_sequences - actual_length
    
    if padding_needed > 0:
        # Left-pad with zeros
        padded_sequence[padding_needed:] = sequence_data[-actual_length:]
    else:
        # Take the last max_sequences if we have more data than needed
        padded_sequence = sequence_data[-max_sequences:]
    
    return padded_sequence

# Example usage:
print("Utility function for inference created!")
print("Use create_inference_sequence() to create padded sequences for prediction")
print("when you have variable amounts of historical data for a player.")

Utility function for inference created!
Use create_inference_sequence() to create padded sequences for prediction
when you have variable amounts of historical data for a player.
